In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

#### Specifying the paths

In [2]:
csv_path = Path(
    r"\\wsl.localhost\Ubuntu-24.04\home\user\OF\attempt_2\results\csv\results.csv"
)
figures_path = Path(
    r"\\wsl.localhost\Ubuntu-24.04\home\user\OF\attempt_2\results/figures"
)
figures_path.mkdir(exist_ok=True, parents=True)

#### Loading in data

In [3]:
force_coeff_df = pd.read_csv(csv_path, parse_dates=["datetime"])
force_coeff_df["cl_cd"] = force_coeff_df["cl"] / force_coeff_df["cd"]
force_coeff_df["cl_cd_abs"] = (force_coeff_df["cl"] / force_coeff_df["cd"]).abs()
force_coeff_df["failed"] = (
    ~force_coeff_df[["no_clipping", "block_mesh", "check_mesh", "simple"]]
).any(axis=1)

df_filtered = force_coeff_df

df_filtered = force_coeff_df.dropna(subset=["cl", "cd"]).copy()
# df_filtered = df_filtered[
#     df_filtered["run_name"] == "5_degree_AoA_fixed_nu_tilda_reduced_yplus"
# ]
# df_filtered = df_filtered[
#     df_filtered["cl_cd_abs"] < 200
# ]
df_filtered["moving_avg"] = df_filtered["cl_cd_abs"].rolling(window=30).mean()

In [ ]:
df_filtered.sort_values(by="cl_cd_abs", ascending=False)
# len(df_filtered)

#### Progress plot

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(
    df_filtered["datetime"],
    df_filtered["cl_cd_abs"],
    label="Lift-drag ratio (Cl/Cd)",
    marker="o",
    color="tab:blue",
    alpha=0.25,
)
plt.plot(
    df_filtered["datetime"],
    df_filtered["cl_cd_abs"].cummax(),
    label="Cumulative max Cl/Cd",
    linestyle="--",
    color="tab:green",
)
plt.scatter(
    force_coeff_df[force_coeff_df["failed"]]["datetime"],
    -force_coeff_df[force_coeff_df["failed"]]["failed"],
    s=10,
    color="tab:red",
    alpha=0.1,
    label="Failed runs",
)

plt.plot(
    df_filtered["datetime"],
    df_filtered["moving_avg"],
    label="Moving average",
    linestyle="-.",
    color="tab:orange",
)

plt.title(f"Lift-drag ratio over time - {len(force_coeff_df)} simulations")
plt.xlabel("Time")
plt.ylabel("$C_l/C_d$ ratio")

# Use a locator to reduce the density of x-ticks
# locator = mdates.HourLocator(interval=1)
# formatter = mdates.DateFormatter("%H:%M")
# plt.gca().xaxis.set_major_locator(locator)
# plt.gca().xaxis.set_major_formatter(formatter)

plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(
    "results/figures/27122024 - OpenFOAM - Lift-drag ratio over time.png", dpi=300
)
plt.show()

#### Attempts at model reduction

In [ ]:
df_subset = df_filtered[
    (df_filtered["cl_cd_abs"] > 5.0) & (df_filtered["cl_cd_abs"] < 1005.0)
]

X = df_subset[["x0", "x1", "x2", "x3", "x4", "x5"]]
y = df_subset["cl_cd"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
percentage_error = np.mean(abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Percentage Error: {percentage_error:.2f}%")

# Dummy method: Mean baseline
y_dummy_pred = [y_train.mean()] * len(y_test)
mse_dummy = mean_squared_error(y_test, y_dummy_pred)
mae_dummy = mean_absolute_error(y_test, y_dummy_pred)
percentage_error_dummy = np.mean(abs((y_test - y_dummy_pred) / y_test)) * 100

print("\nDummy Model Evaluation:")
print(f"Mean Squared Error (Dummy): {mse_dummy}")
print(f"Mean Absolute Error (Dummy): {mae_dummy}")
print(f"Mean Percentage Error (Dummy): {percentage_error_dummy:.2f}%")

# plt.hist((y_test - y_pred) / y_test * 100, bins=100)
plt.hist(abs((y_test - y_pred)), bins=30)
plt.xlabel("Error (MAE)")
plt.ylabel("Bin count")
plt.title("Model reduction with random forest")
plt.grid()

plt.savefig(
    "results/figures/27122024 - OpenFOAM - random forest model reduction.png", dpi=300
)

##### Predicted vs. actual values

In [7]:
pd.set_option("display.max_rows", None)
model_reduction_df = pd.DataFrame({"pred": y_pred, "test": y_test.values}).sort_values(
    by="test", ascending=False
)
model_reduction_df["error"] = model_reduction_df["test"] - model_reduction_df["pred"]
model_reduction_df["percent_error"] = (
    (model_reduction_df["test"] - model_reduction_df["pred"])
    / model_reduction_df["test"]
    * 100
)

#### Forces over time

In [ ]:
force_coeff_df = pd.read_csv(
    r"\\wsl.localhost\Ubuntu-24.04\home\user\OF\attempt_2\custom_runs/aoa\41.05_degree_AoA\postProcessing\forceCoeffs\0\coefficient.dat",
    skiprows=12,
    sep="\t",
)
force_coeff_df.columns = [column_name.strip() for column_name in force_coeff_df.columns]

fig, ax1 = plt.subplots()

# Plot Cl and Cd on the first y-axis
(line1,) = ax1.plot(
    force_coeff_df["# Time"], force_coeff_df["Cl"], label="$C_l$", color="tab:blue"
)
(line2,) = ax1.plot(
    force_coeff_df["# Time"], force_coeff_df["Cd"], label="$C_d$", color="tab:green"
)
ax1.set_xlabel("Time")
ax1.set_ylabel("$C_l$, $C_d$")
# ax1.set_ylim([-0.0005, 0.0005])

# Create a second y-axis for Cl/Cd
ax2 = ax1.twinx()
(line3,) = ax2.plot(
    force_coeff_df["# Time"],
    force_coeff_df["Cl"] / force_coeff_df["Cd"],
    label="$C_l/C_d$",
    color="tab:red",
)
ax2.set_ylabel("$C_l/C_d$")
ax2.set_ylim([0, 50])

# Combine legends
lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="upper right")

plt.title("Forces over time")
plt.tight_layout()
plt.savefig(
    "results/figures/01012025 - OpenFOAM - oscillatory solution to lift-drag ratio - 41.05 deg AoA.png",
    dpi=300,
)
plt.show()

#### Lift-drag vs angle-of-attack

In [ ]:
aoa_df = pd.read_csv("results/csv/aoa_results.csv")

aoa_df["aoa"] = aoa_df["run_name"].str.extract(r"(\d+\.\d+)_degree_AoA").astype(float)
aoa_df = aoa_df[aoa_df["cl"] >= 0]

aoa_df = aoa_df.sort_values(by="aoa")

plt.plot(aoa_df["aoa"], aoa_df["cl"] / aoa_df["cd"], "-x")
plt.xlabel("Angle-of-attack")
plt.ylabel("$C_l/C_d$")

plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.1f}°"))

plt.title("Lift-drag ratio")
plt.grid()
plt.savefig(
    "results/figures/01012025 - OpenFOAM - lift-drag ratio for angles of attack - random airfoil.png",
    dpi=300,
)

#### Lift-drag vs velocity

In [ ]:
velocity_df = pd.read_csv("results/csv/velocity_results.csv")

velocity_df["velocity"] = (
    velocity_df["run_name"].str.extract(r"(\d+\.\d+)_ms").astype(float)
)
velocity_df = velocity_df[velocity_df["cl"] >= 0]

velocity_df = velocity_df.sort_values(by="velocity")

plt.plot(velocity_df["velocity"], velocity_df["cl"] / velocity_df["cd"], "-x")
plt.xlabel("Velocity (m/s)")
plt.ylabel("$C_l/C_d$")

plt.title("Lift-drag ratio")
plt.grid()
plt.savefig(
    "results/figures/01012025 - OpenFOAM - lift-drag ratio for velocities - random airfoil.png",
    dpi=300,
)
